## GNNs para Séries Temporais

Séries temporais representam dados sequenciais que mudam ao longo do tempo, como medições de sensores, valores de ações ou dados climáticos. Tradicionalmente, abordagens como redes neurais recorrentes (RNNs) e modelos baseados em convoluções temporais (TCNs) têm sido utilizadas para capturar a dinâmica temporal. No entanto, quando a série temporal está associada a uma rede (como sensores em uma rede elétrica, estações meteorológicas ou tráfego de uma rede viária), o uso de **Graph Neural Networks (GNNs)** oferece uma maneira poderosa de capturar tanto as **relações espaciais** quanto as **dependências temporais**.

### Representando Séries Temporais em Grafos

Para utilizar GNNs em séries temporais, os dados podem ser representados como um **grafo dinâmico**. A ideia principal é tratar cada instante de tempo como uma sequência de estados do grafo. Os nós representam variáveis (ou locais) na série temporal e as arestas entre os nós capturam as dependências espaciais ou relacionais. Aqui estão algumas formas de definir um problema de séries temporais em termos de grafos:

1. **Nós como variáveis**: Cada nó do grafo representa uma variável ou localização específica, como sensores em uma rede, onde os dados da série temporal de cada nó mudam com o tempo.
2. **Arestas como dependências espaciais**: As arestas podem representar a proximidade física ou correlações entre as variáveis. Por exemplo, em uma rede de tráfego, as estradas conectam pontos de tráfego (nós), criando dependências espaciais.
3. **Dimensão temporal**: Cada nó tem uma sequência de valores ao longo do tempo, e a evolução temporal de cada nó precisa ser capturada.

### Definindo GNNs para Séries Temporais

A abordagem mais comum para aplicar GNNs em séries temporais é combinar **modelos GNN** que capturam as **relações espaciais** com modelos que capturam a **dinâmica temporal**. Existem duas maneiras principais de fazer isso:

#### 1. **Spatial-Temporal Graph Neural Networks (ST-GNNs)**

Esses modelos são projetados para capturar tanto as dependências espaciais quanto temporais. A arquitetura básica é a seguinte:

- **Dependências Espaciais**: Uma GNN é usada para capturar as relações espaciais entre os nós a cada instante de tempo. Isso é feito utilizando uma convolução de grafos em cada frame temporal para aprender a relação entre os nós conectados.
  
- **Dependências Temporais**: Um modelo como uma rede recorrente (LSTM, GRU) ou convoluções temporais (TCN) é aplicado para capturar a evolução temporal da sequência.

A formulação geral de uma ST-GNN é:
$$
H^{(t+1)} = f_{\text{temp}} \left( \sigma \left( f_{\text{spat}}(A, H^{(t)}) \right) \right),
$$
onde:
- $f_{\text{spat}}$ é a GNN que aprende as dependências espaciais.
- $f_{\text{temp}}$ captura as dependências temporais (pode ser uma LSTM ou TCN).
- $A$ é a matriz de adjacência representando a conectividade do grafo.
- $H^{(t)}$ são as features em cada tempo $t$.
- $\sigma$ é uma função de ativação, como ReLU.

#### 2. **Graph Convolutional Recurrent Networks (GCRNs)**

Essa abordagem integra a convolução de grafos diretamente dentro de uma arquitetura recorrente. Em vez de tratar o espaço e o tempo de maneira separada, o modelo aplica uma convolução de grafos em cada passo recorrente.

O GCRN pode ser formulado como:

$$
h^{(t+1)} = \text{RNN} \left( \text{GNN}(A, h^{(t)}), x^{(t+1)} \right),
$$

onde:
- $h^{(t)}$ é o estado oculto do RNN no tempo $t$.
- $\text{GNN}(A, h^{(t)})$ aplica a convolução de grafos sobre o estado oculto, integrando a estrutura do grafo.
- $x^{(t+1)}$ são os novos dados de entrada no tempo $t+1$.

### Aplicações de GNNs em Séries Temporais

1. **Previsão de Tráfego**: Em redes de transporte, cada ponto de tráfego pode ser modelado como um nó e as estradas como arestas. A GNN captura as dependências espaciais entre diferentes locais, enquanto uma RNN ou TCN captura a evolução do tráfego ao longo do tempo.
  
2. **Monitoramento de Redes Elétricas**: Em sistemas de energia, sensores distribuídos em diferentes locais coletam dados ao longo do tempo. A GNN modela as relações entre os sensores (por exemplo, proximidade física ou conexões elétricas), enquanto o componente temporal captura a evolução dos dados de sensores.

3. **Análise de Saúde**: Em aplicações de saúde, como monitoramento de pacientes em hospitais, GNNs podem ser usadas para capturar a relação entre diferentes parâmetros fisiológicos (nós), enquanto modelos temporais capturam a evolução dos sinais vitais ao longo do tempo.

### Conclusão

Aplicar GNNs a séries temporais permite capturar tanto as dependências espaciais entre variáveis ou locais quanto as dependências temporais que regem a evolução dos dados. Modelos como ST-GNNs e GCRNs integram essas duas dimensões, proporcionando uma maneira eficaz de lidar com dados que variam no espaço e no tempo.
